# UDACITY Designing Your First Workflow - Tying it All Together Solution

## Exercise: Create the Lambda Function

In [ ]:
import json
import boto3
import time

client = boto3.client('stepfunctions')

definition = """{
  "StartAt": "SageMaker pre-processing step",
  "States": {
    "SageMaker pre-processing step": {
      "Resource": "arn:aws:states:::sagemaker:createProcessingJob.sync",
      "Parameters": {
        "ProcessingJobName": "test-joblambda-preprocess",
        "ProcessingInputs": [
          {
            "InputName": "input-1",
            "AppManaged": false,
            "S3Input": {
              "S3Uri": "s3://udacity-sagemaker-solutiondata2021/l3e1/reviews_Musical_Instruments_5.json.zip",
              "LocalPath": "/opt/ml/processing/input",
              "S3DataType": "S3Prefix",
              "S3InputMode": "File",
              "S3DataDistributionType": "FullyReplicated",
              "S3CompressionType": "None"
            }
          },
          {
            "InputName": "code",
            "AppManaged": false,
            "S3Input": {
              "S3Uri": "s3://udacity-sagemaker-solutiondata2021/l3e3/HelloBlazePreprocess.py",
              "LocalPath": "/opt/ml/processing/input/code",
              "S3DataType": "S3Prefix",
              "S3InputMode": "File",
              "S3DataDistributionType": "FullyReplicated",
              "S3CompressionType": "None"
            }
          }
        ],
        "ProcessingOutputConfig": {
          "Outputs": [
            {
              "OutputName": "train_data",
              "AppManaged": false,
              "S3Output": {
                "S3Uri": "s3://sagemaker-us-west-2-565094796913/hello_blaze_train_scikit",
                "LocalPath": "/opt/ml/processing/output/train",
                "S3UploadMode": "EndOfJob"
              }
            },
            {
              "OutputName": "test_data",
              "AppManaged": false,
              "S3Output": {
                "S3Uri": "s3://sagemaker-us-west-2-565094796913/hello_blaze_test_scikit",
                "LocalPath": "/opt/ml/processing/output/test",
                "S3UploadMode": "EndOfJob"
              }
            }
          ]
        },
        "AppSpecification": {
          "ImageUri": "246618743249.dkr.ecr.us-west-2.amazonaws.com/sagemaker-scikit-learn:0.20.0-cpu-py3",
          "ContainerEntrypoint": [
            "python3",
            "/opt/ml/processing/input/code/HelloBlazePreprocess.py"
          ]
        },
        "RoleArn": "arn:aws:iam::565094796913:role/execution_role",
        "ProcessingResources": {
          "ClusterConfig": {
            "InstanceCount": 1,
            "InstanceType": "ml.m5.large",
            "VolumeSizeInGB": 30
          }
        }
      },
      "Type": "Task",
      "Next": "SageMaker Training Step"
    },
    "SageMaker Training Step": {
      "Resource": "arn:aws:states:::sagemaker:createTrainingJob.sync",
      "Parameters": {
        "AlgorithmSpecification": {
          "TrainingImage": "433757028032.dkr.ecr.us-west-2.amazonaws.com/blazingtext:1",
          "TrainingInputMode": "File"
        },
        "OutputDataConfig": {
          "S3OutputPath": "s3://udacity-sagemaker-solutiondata2021/l3e3/workflow_output"
        },
        "StoppingCondition": {
          "MaxRuntimeInSeconds": 360000
        },
        "ResourceConfig": {
          "InstanceCount": 1,
          "InstanceType": "ml.m5.large",
          "VolumeSizeInGB": 30
        },
        "RoleArn": "arn:aws:iam::565094796913:role/execution_role",
        "InputDataConfig": [
          {
            "DataSource": {
              "S3DataSource": {
                "S3DataType": "S3Prefix",
                "S3Uri": "s3://sagemaker-us-west-2-565094796913/hello_blaze_train_scikit",
                "S3DataDistributionType": "FullyReplicated"
              }
            },
            "ContentType": "text/plain",
            "ChannelName": "train"
          },
          {
            "DataSource": {
              "S3DataSource": {
                "S3DataType": "S3Prefix",
                "S3Uri": "s3://sagemaker-us-west-2-565094796913/hello_blaze_test_scikit",
                "S3DataDistributionType": "FullyReplicated"
              }
            },
            "ContentType": "text/plain",
            "ChannelName": "validation"
          }
        ],
        "HyperParameters": {
          "mode": "supervised"
        },
        "TrainingJobName": "test-lambda-train"
      },
      "Type": "Task",
      "End": true
    }
  }
}"""

def lambda_handler(event, context):
    
    client.update_state_machine(definition=definition, stateMachineArn='arn:aws:states:us-west-2:565094796913:stateMachine:SageMakerProcessingWorkflow7')
    time.sleep(5)
    response = client.start_execution(input='{}', name='joblambda2', stateMachineArn='arn:aws:states:us-west-2:565094796913:stateMachine:SageMakerProcessingWorkflow7')
    
    return response
